# Step 0) Load Package & config file

In [25]:
## 함수 로딩  
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 105)  # or 1000
pd.set_option('display.max_rows', 2000)    # or 1000
import time 
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import argparse
import multiprocessing
from functools import partial
from tqdm.contrib.concurrent import process_map  # or thread_map
from tqdm import tqdm
import ray
import random
from matplotlib import pyplot as plt
import warnings
import logging # This allows for seeing if the model converges. A log file is created.
import pickle,psutil,shutil

# log
import logging
import logging.config
import traceback

## 전처리 모듈
from sklearn.preprocessing import Normalizer
from itertools import chain

## Gensim 모듈
import gensim
from gensim.models import LdaModel

## 후처리 모듈
import re,os,sys
from functools import reduce
sys.path.append('/analytics/mdap/src/codeshare/')
import redshift_dbconnect as rs_connect
import data_management as dm
import delete_all_object_in_bucket as rm_all_bucket

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import types
from sqlalchemy import inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric

import boto3
from boto3.s3.transfer import TransferConfig
from io import StringIO,BytesIO
import psycopg2
from IPython.display import display
import gc
import configparser

conf_dir = '/analytics/mdap/src/conf/config.ini'
cfg = configparser.ConfigParser(interpolation=configparser.ExtendedInterpolation())
cfg.read(conf_dir)

#---------------------------------------------------------
# ■ setting configuration (db_connectionm, load_model)
#---------------------------------------------------------
global HOST,PORT,DB_ID,DB_PW,MODEL_DIR,MODEL_NAME,AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_DEFAULT_REGION, BUCKET_NAME,IAM_ARN,EXCEP_ALCOHOL_TEAM_CD,DATA_DIR,HOME_DIR,LOG_CONFIG
DBNAME                   = cfg['dbconnect_opt']['dbname']
HOST                     = cfg['dbconnect_opt']['host']
PORT                     = int(cfg['dbconnect_opt']['port'])
USER_ID                  = cfg['dbconnect_opt']['user_id']
PASSWORD                 = cfg['dbconnect_opt']['password']
AWS_ACCESS_KEY_ID        = cfg['s3connect']['aws_access_key_id']
AWS_SECRET_ACCESS_KEY    = cfg['s3connect']['aws_secret_access_key']
AWS_DEFAULT_REGION       = cfg['s3connect']['region']
BUCKET_NAME              = cfg['s3connect']['bucket']
IAM_ARN                  = cfg['s3connect']['ima_arn'] 
EXCEP_ALCOHOL_TEAM_CD    = cfg['Recommender']['excep_alcohol_team_cd']
DATA_DIR                 = cfg['Data']['data_dir']
MODEL_DIR                = cfg['Recommender']['model_dir']
HOME_DIR                 = cfg['common']['home_dir']
LOG_CONFIG               = cfg['common']['log_config']
#==========================================================
# ■ Argument parse
#==========================================================

## -----------------------------------------
# parse init args
## -----------------------------------------
import argparse
# parser = argparse.ArgumentParser(description = 'LDA Recommender System ')
# parser.add_argument('--target_date', help = '분석 기준일')
# args = parser.parse_args()

target_date = '2022-11-01'
today_date = '20221101'

module_name = 'M1_DAILY_BATCH_PROD_RECOM_APPLY.py'
model_name = 'M1_DAILY_BATCH_PROD_RECOM_APPLY'
model_id = 'RECOM'

# Load Functions

In [6]:
# Step 1) Load functions
def file_search(file_dir, pattern) :    
    files = os.listdir(file_dir)
    p = re.compile(pattern) 

    f_list = []
    for f in files : 
        out = p.match(f)
        if out != None : f_list.append(f)
        else : pass
        
    f_list = np.sort(f_list) ## 오름차 순으로 Sorting

    return f_list

# Step 2) Definition of global variable and Load model
def LDA_Model_load(model_path):
    m_list = file_search(file_dir = model_path, pattern = "^lda_model_[0-9]+[\.]model$")
    model_target_nm = m_list[-1] # Max 값으로 지정. 가장 최신 일자 모델을 끌고 옴
    m_path = model_path +model_target_nm
    lda_model = LdaModel.load(m_path) 
    
    prdt_mst_list = file_search(file_dir = model_path, pattern = "^prdt_mst_[0-9]+[\.]ftr$")
    prdt_mst_target_nm = prdt_mst_list[-1] # Max 값으로 지정. 가장 최신 일자 모델을 끌고 옴
    prdt_map = pd.read_feather(model_path + prdt_mst_target_nm)
    print(f'==== Load LDA Model Name : {model_target_nm} ====')
    return lda_model,prdt_map,model_target_nm


def LDA_Model_seach_load(model_nm):
    m_path = MODEL_DIR +model_nm
    lda_model = LdaModel.load(m_path) 
    print(f'==== Load LDA Model Name : {m_path} ====')
    return lda_model


# Step 3) Pre-Processing
def data_preparation(df) : 
    start_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    start_tt = time.time()

    ## TF Transformation
    tf, idf, tfidf = tf_idf_matrix(df)

    ## Product Mapping
    prdt_cd = list(df['L3_CD'].unique())
    prdt_cd_df = prdt_map[prdt_map['L3_CD'].isin(prdt_cd)].sort_values(by = 'L3_CD', ascending = True)

    ## Corpus 생성
    corpus = tf.apply(row_to_tuple, mst = prdt_cd_df, axis = 1)

    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
          " || TF-IDF : Data Preprocessing is completed.",
          "(elapsed time :", np.round((time.time() - start_tt) / 60, 2), "mins)")

    return prdt_cd_df, corpus

def row_to_tuple(a, mst) : 
    a_flt = a[a > 0]
    a_list = a_flt.index.tolist()

    c = mst.L3_CD 
    c = c[c.isin(a_list)].index.tolist()

    return list(zip(c,a_flt.tolist()))

def tf_idf_matrix(df) :
    ## document length
    N = len(df['INTG_CUST_NO'].unique())

    ## TF Matrix 변환 
    tf = pd.pivot_table(df, values = 'DT_CNT', index = 'INTG_CUST_NO', columns = 'L3_CD')
    tf = tf.fillna(0)

    ## IDF Matrix 계산 
    idf = df.groupby(['L3_CD']).sum()['DT_CNT']
    idf = np.log((N+1)/(idf + 1)) + 1

    # TF-IDF Matrix 계산
    tfidf = tf.mul(idf)

    # L2 정규화 Term
    tfidf.iloc[:,:] = Normalizer(norm = 'l2').fit_transform(tfidf)

    print("          tf.shape :",tf.shape, 
          "idf.shape :",idf.shape, "tfidf.shape :", tfidf.shape)

    return tf, idf, tfidf    

def file_search(file_dir, pattern) :    
    files = os.listdir(file_dir)
    p = re.compile(pattern) 

    f_list = []
    for f in files : 
        out = p.match(f)
        if out != None : f_list.append(f)
        else : pass
        
    f_list = np.sort(f_list) ## 오름차 순으로 Sorting

    return f_list

def Save_corpus(dt_corpus,anl_dt):
    path =f'{MODEL_DIR}'+'corpus/'
    corpus_file_nm = 'corpus_' + anl_dt + '.pkl'
    corpus_file = open(path + corpus_file_nm, "wb")
    pickle.dump(dt_corpus, corpus_file)
    corpus_file.close()
    print(f'====== Corpus save : {corpus_file} ======')
    
def Load_corpus(corpus_nm):
    path =f'{MODEL_DIR}'+'corpus/'
#     corpus_nm = 'corpus_2022-02-02.pkl'
    corpus = pickle.load(open(path+corpus_nm, "rb"))
    return corpus

# Step 4) Logic1 - Cust*Topic distribution
## 신규 토픽 생성 함수 
def list_converter(lst, topic_len) : 
    n_iter = int(len(lst) / topic_len)
    for idx in range(n_iter) : 
        yield lst[topic_len*idx : topic_len*(idx+1)]
        idx += 1 # if idx % 100000 == 0 : print(idx)

def Topic_df_converter(lda_model, corpus) : 

    global target_date
    global model_target_nm

    start = time.time()

    ## Step 1. gensim document topic 을 DF 로 변환
    ## 속도를 위해 CSR Matrix 활용
    ## ----------------------------------------------
    all_topics = lda_model.get_document_topics(corpus, minimum_probability=None)
    print("          Topic_df_converter :","Step 1-1. Loading topic is completed..",np.round((time.time() -start)/60,2),"mins")

    all_topics_csr = gensim.matutils.corpus2csc(all_topics)
    print("          Topic_df_converter :","Step 1-2. Sparse Matrix Coversion is completed..",np.round((time.time() -start)/60,2),"mins")

    all_topics_numpy = all_topics_csr.T.toarray()
    print("          Topic_df_converter :","Step 1-3. Sparse Matrix to Array Transformation is completed..",np.round((time.time() -start)/60,2),"mins")

    df = pd.DataFrame(all_topics_numpy) # all_topics_df
    print("          Topic_df_converter :","Step 1-4. Gensim object --> DF conversion is completed..",
          np.round((time.time() -start)/60,2),"mins")

    ## Step 2. DF 내 element 를 내림차순 Sorting 하여 Tuple 형태로 변환
    ## 변환된 Tuple 을 다시 multidimensional list 로 변환하여 DF 준비
    ## ----------------------------------------------
    topic_col_argsort = df.columns.values[np.argsort(-df.values, axis = 1)]
    topic_val_argsort = -np.sort(-df.values, axis = 1)

    topic_col_hstack = np.hstack(topic_col_argsort)
    topic_val_hstack = np.hstack(topic_val_argsort)
    topic_tuple = [(i,j) for i,j in zip(topic_col_hstack, topic_val_hstack)]

    out = list(list_converter(topic_tuple, len(df.columns)))

    print("          Topic_df_converter :","Step 1-5. Tuple & dimension changes are completed..",
          np.round((time.time() -start)/60,2),"mins")

    ## Step 3. 데이터프레임화 & 결과 테이블 리턴
    ## ----------------------------------------------
    Topic_df= pd.DataFrame(out, index = corpus.index)
    Topic_df.columns = ['topic'+str(i) for i in range(len(df.columns))]
    Topic_df = Topic_df.reset_index().rename(columns = {"index":"cust_id"})

    Topic_df.columns = [col.lower() for col in Topic_df.columns]
#     Topic_df = Topic_df.set_index('cust_id')

    ## Step 4. sql 용 dF 준비
    df_sql = df.copy()
    df_sql.columns = ['topic_no'+str(i) for i in df_sql.columns ]
    df_sql['intg_cust_no'] = corpus.index
    df_sql['crtn_dt'] = target_date
    df_sql['model_version'] = model_target_nm

    col_seq = ['crtn_dt','intg_cust_no','model_version'] + ['topic_no' + str(i) for i in range(100)]
    df_sql = df_sql[col_seq]

    print("          Topic_df_converter :","Step 1-6. DF transformation is completed..",
          np.round((time.time() -start)/60,2),"mins")

    ## Print progress 
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
          " || Topic DF generation is completed.",
          "(elapsed time :", np.round((time.time() - start) / 60, 2), "mins)")

    return Topic_df, df_sql



## Additional : 토픽 해석용 도출 & Topic_df Top N토픽 필터링 DF 도출
def topic_distribution(n, lda_model, Topic_df, nword) :

    """
    parameter 
    -------------
    @ n : Topic 수
    @ lda_model : 대상 LDA 모델
    @ Topic_df : 토픽의 Dataframe
    @ nword : 토픽 별 상품 대상 Top N
    """
    ## --------------------------------------------------------
    ## A.Top-N 만큼 해당하는 토픽을 뽑아 데이터 프레임 생성
    ## --------------------------------------------------------

    ## Step 1. Topic_df 중 토픽에 해당하는 유효 칼럼만 추출
    col_list = []
    for i in Topic_df.columns : 
        pattern = re.compile('^topic[0-9]+$')
        out = pattern.search(str(i))
        if out != None : 
            col_list.append(i)
        else : 
            pass

    ## Step 2. Topic_df 중 Top N 개 토픽만 추출하여 데이터프레임화
    col_argmax = [int(i.replace('topic','')) for i in col_list]
    top_n = np.array(col_argmax).argsort()[:n]
    top_n_col = [col_list[i] for i in top_n]

    df_flt = Topic_df[top_n_col]
    df_flt_export = Topic_df[['intg_cust_no']+top_n_col] ## 타 분석의 input 용도로 활용
    # display(df_flt.head())

    ## --------------------------------------------------------
    ## B.토픽별 Count 비중 (Top1 ~ Top N 까지의 누적 비중)
    ## --------------------------------------------------------

    ## Step 1. 토픽을 갖고 있는 고객 수 계산 
    N_valid = len(df_flt[~df_flt.iloc[:,0].isna()])
    N_all = len(df_flt.iloc[:,0])
    print("The No. of Customers who has topic :",N_valid,"/",N_all )
    print("The No. of Customers who don't have topic :",N_all-N_valid,"/",N_all)

    ## Step 2. 토픽 칼럼을 합쳐 list 화 
    array_list = []
    for i in range(n) : 
        array_list.append(df_flt.iloc[:,i])

    # Step 3. Top 1 부터 Top N 까지의 누적 비중 합 구해주기
    df_list = []
    for k in range(n) : 
        if k == 0 :
            t = array_list[0]
        else : 
            t = reduce(lambda x, y: np.hstack([x, y]), array_list[:k+1])
            t = pd.Series(t)

        topic_ls = t[~t.isna()].apply(lambda x : x[0])
        topic_grp = [[i, np.array(topic_ls == i).sum()] for i in np.unique(topic_ls)]
        topic_grp = pd.DataFrame(topic_grp, columns = ['topic_no','topic_cnt_'+str(k)])
        topic_grp['topic_ratio_'+str(k)] = round(topic_grp['topic_cnt_'+str(k)] *100 / N_valid ,5) # Top1_df['topic_cnt'].sum() 

        df_list.append(topic_grp)

    # Step 4. Merge
    df_merge = reduce(lambda x,y: pd.merge(x,y, how = 'left', on = 'topic_no'),  df_list) 

    ## ---------------------------------------
    ## C. 토픽 별 상위 N개개 상품 가중치 도출
    ## ---------------------------------------
    topic_keyword = lda_model.print_topics(num_topics = 100, num_words = nword)
    topic_keyword_df = pd.DataFrame(topic_keyword, columns = ['topic_no','topic_keyword_weight'])

    ## ---------------------------------------
    ## D. 데이터 결합
    ## ---------------------------------------
    df_final = pd.merge(df_merge, topic_keyword_df, how = 'left', on = 'topic_no')
    # df_final = df_final.sort_values(by = 'topic_cnt', ascending = False)
    # display(df_final)

    return df_flt, df_final


def DB_Connection(DBNAME = DBNAME, HOST = HOST, PORT = PORT, USER_ID = USER_ID,PASSWORD=PASSWORD) :
    connection_info = "dbname='{}' host='{}' port={} user='{}' password='{}'".format(DBNAME, HOST, PORT, USER_ID, PASSWORD)
    conn = psycopg2.connect(connection_info)
    return conn

def init_connection() :
#     engine = create_engine(f'redshift:///?User={USER_ID}&;Password={PASSWORD}&Database={DBNAME}&Server={HOST}&Port={PORT}', echo = False)
    engine = create_engine(f'redshift+psycopg2://{USER_ID}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')
    conn = engine.connect()
    return (conn, engine) 

def File_exists_delete(file):
    if(os.path.exists(file) and os.path.isfile(file)):
        os.remove(file)
        print("file deleted")
    else:
        print("file not found")

## 4-3) Calculate Logic1 probwt
### - 고객xtopic 분포 추출 ( Top 10)
### - 고객xtopicx소분류 확률 계산후 상품별 토픽분포 x 해당 토픽내 상품분포계산 후 sum

def Schema_inspector(schema,tbl_name):
    schema_df = rs_connect.query_request(f''' select distinct table_name from information_schema.tables where table_schema = '{schema}' ''');
    if tbl_name.lower() in schema_df.TABLE_NAME.str.lower().tolist() : 
        rs_connect.cud_request(f'''DROP TABLE {schema}.{tbl_name} ''')
        print(f'''{schema}.{tbl_name} table is drop''')
    else : 
        print(f''' Table {schema}.{tbl_name} doesn't exist  ''')

## -----------------------------------------------
## 함수 세팅
## -----------------------------------------------
def Upload_parquet_to_s3(result_df,filename):
    s3 = boto3.client("s3",
                          aws_access_key_id = AWS_ACCESS_KEY_ID,
                          aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                          region_name = AWS_DEFAULT_REGION)    
    parquet_buffer = BytesIO()
    result_df.to_parquet(parquet_buffer, index=False)
    s3.put_object(Bucket=BUCKET_NAME, Key=filename, Body=parquet_buffer.getvalue())
    print('Upload parquet file to s3')

    
def Select_Logic1_baske_df(anl_dt):
    sql = f''' 
    WITH TMP AS (
            -- 최근 7일간 구매이력 있는 고객만 필터링
            SELECT INTG_CUST_NO, L3_CD, L3_NM, ANL_DT, DT_CNT, 
                   DENSE_RANK() OVER (PARTITION BY INTG_CUST_NO ORDER BY ANL_DT DESC) AS CUST_RANK
            FROM   LMDAPT3_DB.F_ANL_PROD_RECOM_APPLY
            WHERE  ANL_DT BETWEEN TO_DATE(dateadd(day,-6,'{anl_dt}'),'YYYY-MM-DD') AND DATE('{anl_dt}')
        ), 
         TMP2 AS (
            SELECT  L3_CD
                   ,L3_NM
                   ,SUM(DT_CNT) AS DT_CNT_ALL
              FROM TMP 
             WHERE CUST_RANK = 1 -- 최근 7일 이내 2번 이상 내방한 고객들의 구매이력 중 최신 이력만 필터링

          GROUP BY L3_CD, L3_NM
        )
        SELECT   A.L3_CD
                ,A.L3_NM
                ,A.DT_CNT_ALL
                ,B.CUST_CNT_ALL
                ,cast(A.DT_CNT_ALL as DECIMAL) / CAST(B.CUST_CNT_ALL as DECIMAL) AS DT_CNT_RT_ALL -- 최근 7일간 전체 고객의 90일간 품목별 구매 비율
          FROM  TMP2 A,
            (SELECT COUNT(DISTINCT INTG_CUST_NO) AS CUST_CNT_ALL FROM TMP) B
    '''
    df = rs_connect.query_request(sql)
    return df

# @ray.remote
def recommender_logic_1(row_id) :  
    global lda_model_iter
    global corpus_iter
    global Topic_df
    global prdt_map
    global topN

    lda_model = lda_model_iter
    corpus = corpus_iter.copy() 
    prdt_mst = prdt_map.copy()


    ##  고객 기구매이력 (Corpus_df) 과 상품마스터 (dict_df)를 붙임
    corpus_df = pd.DataFrame(corpus[row_id], columns = ['id','dt_cnt'])
    prdt_mst = prdt_mst.rename(columns = {"row_id":"id","L3_CD":"l3_cd","L3_NM":"l3_nm"})

    try : 
        ## 고객 별 Topic list 를 X 에 저장
        X = Topic_df.iloc[:,1:].iloc[row_id].apply(lambda x : [int(x[0]),float(x[1])])[0:10]

        Xa = X[X.map(lambda y: y[1]) > 0].tolist() 
        X = [i[0] for i in Xa] 

        ## 토픽별로 상품 가중치 DataFrame 화
        X_list = [] 
        ## 고객의 상위 TOP10토픽
        for j,i in enumerate(X) :
            """토픽별 상품 Top 30 개만 가져옴
            """
            X_term = lda_model.get_topic_terms(int(i), topn = 30) 
            X_df = pd.DataFrame(X_term, columns = ['id',"topic"+str(j)+"_dist"])
            X_list += [X_df]

        X_merge = reduce(lambda x, y: pd.merge(x, y, how = 'outer', on = 'id'), X_list)

        X_final = pd.merge(corpus_df, X_merge, how = 'outer', on = 'id')
        X_final = pd.merge(prdt_mst, X_final, how = 'right', on = 'id')
        X_final['intg_cust_no'] = corpus.index[row_id]

    except : 
        """
        구매 이력이 거의 없는 케이스 예외 처리
        토픽이 생성되지 않는 고객의 경우 구매 이력만 가지고 떨굼
        """
        Xa = [(9999,1)]
        X_final = pd.merge(corpus_df, prdt_mst, how = 'left', on = 'id')
        X_final['intg_cust_no'] = corpus.index[row_id]

    return recommender_logic_1_ranking(topic_dist = Xa, df = X_final)

def recommender_logic_1_ranking(topic_dist, df) : 
    """
    params
    ----------
    @ topic_dist : 고객 별 토픽 분포 확률 
    @ df : 고객 X 토픽 별 상품 및 구매이력 Dataframe
    @ topN : 상위 몇개의 Topic만 대상으로 할것인지 
    """
    global target_date,cust_id_repair

    cust_id_repair = []

    try : 
        ## 로직 1 : 고객 별 토픽확률 X 토픽별 상품확률 곱 합산 방식 
        X_col = df.columns[df.columns.str.contains('topic')] # sales['컬럼명'].str.contains('검색하고자 하는 문자열')
        wt    = [i[1] for i in topic_dist] ## 토픽 분포 확률 추출
        X_wt  = df[X_col] * wt

        df['prob_wt']      = X_wt.sum(axis=1)
        df['prob_wt_rank'] = df['prob_wt'].rank(method = 'dense', ascending = False)
        ## LIFT 로직 : 해당 상품의 전체판매비중 대비 로직1계산(확률곱) 상대비중

        ## ----------------
        df = pd.merge(df, basket_agg_df, how = 'left', on = ['l3_cd','l3_nm'])
        df['lift'] = df['prob_wt'] / df['dt_cnt_rt_all']
        df['lift_rank'] = df['lift'].rank(method = 'dense', ascending = False)
        ## ----------------

        ## TOPN 토픽만 대상으로 확률곱 계산하고, LIFT 랭킹 구함
        X_wt_topN = df[X_col].iloc[:,:topN] * wt[:topN] 
        df['prob_wt_top'+str(topN)] = X_wt_topN.sum(axis=1)
        df['lift_top'+str(topN)] = df['prob_wt_top'+str(topN)] / df['dt_cnt_rt_all']
        df['lift_rank_top'+str(topN)] = df['lift_top'+str(topN)].rank(method = 'dense', ascending = False)

        ## -----------------

    except Exception as e: 
        print(f'except : {e}')
        cust_id_repair += [df.intg_cust_no[0]]

        """ 구매 이력이 거의 없는 케이스 예외 처리
             >> Xa = [(9999,1)] 인 경우 처리 로직에 해당

            이 경우, 최종 로직에서 장바구니의 랭킹으로 처리
            flag 는 prob_wt = 9 , lift = 9999999999 로 처리 
        """
        df['dt_cnt_all'] = [None] * len(df) 
        df['cust_cnt_all'] = [None] * len(df)
        df['dt_cnt_rt_all'] = [None] * len(df)

        df['lift'] = 9999999999
        df['lift_rank'] = [None] * len(df)
        df['prob_wt']   = 9 
        df['prob_wt_rank'] = [None] * len(df)   

        ## Top 5 토픽 리프트
        df['prob_wt_top'+str(topN)] = [None] * len(df)
        df['lift_top'+str(topN)] = [None] * len(df)
        df['lift_rank_top'+str(topN)] = [None] * len(df)
        ## -----------------
        
    try:
        
        df.columns = [col.lower() for col in df.columns]
        df['anl_dt'] = datetime.datetime.strptime(target_date, '%Y-%m-%d').strftime('%Y-%m-%d') 

        # --------------------------------------------------------------------------
        # 속도를 위하여 Top 10 토픽까지만 필터 --> 정형화시킴
        # 정형화시킨 데이터프레임을 concat 형태가 아닌 append 형태로 바로 적재 진행
        # --------------------------------------------------------------------------

        # """ 데이터 프레임용 칼럼 순서 결정 
        # """
        col_common = ["anl_dt","intg_cust_no","id","l3_cd", "l3_nm","dt_cnt","prob_wt","prob_wt_rank",
                      "dt_cnt_all","cust_cnt_all","dt_cnt_rt_all","lift","lift_rank",
                      "prob_wt_top"+str(topN),"lift_top"+str(topN),"lift_rank_top"+str(topN)] 
        pattern = re.compile(r"^topic[0-9]+[\_]dist$")

        col_topic = [col for col in list(df.columns) if pattern.match(col) != None ]

        # """ 고객 별 토픽 수가 10개 미만인 경우: 기존 토픽을 넣고 나머지는 dummy 처리
        #     고객 별 토픽 수가 10개 이상인 경우: 상위 토픽 10개만 짤라서 가져옴
        # """
        if len(col_topic) < 10 : 
            col_topic = [ "topic" + str(i) + "_dist" for i in range(0,len(col_topic))]
            col_seq = col_common + col_topic
            df_flt = df[col_seq]

            col_topic_dummy = [ "topic" + str(i) + "_dist" for i in range(len(col_topic),10)]
            for col_dummy in col_topic_dummy :
                df_flt[col_dummy] = [None] * len(df_flt)

        else : 
            col_topic = [ "topic" + str(i) + "_dist" for i in range(0,10)]
            col_seq = col_common + col_topic
            df_flt = df[col_seq]

        df_flt = df_flt.where(pd.notnull(df_flt), None)
#         rs_connect.replace_table(df_flt,'lmdapwk_anl_db.f_anl_logic1_probwt_result',True)
#         listToWrite = df_flt.to_dict(orient = 'records')
        
    except Exception as e :
        pritn(f' Logic 1 Result Error : {e}')
    return df_flt#cust_id_repair #df_flt #listToWrite 

def multiprocesser_logic1_tqdm(worker, arg_list, n_core,filename) :  
    start_time = time.time()
    result_list = []
    
    print("          Multiprocessing is getting started...", np.round((time.time() - start_time)/60,2),"mins")
    if __name__ == '__main__' : 
        try:
            p = multiprocessing.Pool(processes = n_core)
            p.deamon = True
            for result in tqdm(p.imap_unordered(func = worker, iterable = arg_list), total = len(arg_list)) :
                result_list.append(result)
#                 result.to_csv(target, index=None, sep=',', mode='a', header=True)
#             result_list += tqdm(p.imap_unordered(func = worker, iterable = arg_list), total = len(arg_list))
            print(1)
            p.close()
            print(2)
#             p.terminate()
            print(3)
            time.sleep(2)
            p.join()
            print(4)
            time.sleep(2)
            result_df = pd.concat(result_list)
            result_df = result_df.fillna(0)
            result_df.loc[:,['anl_dt','intg_cust_no','l3_cd','l3_nm']] = result_df[['anl_dt','intg_cust_no','l3_cd','l3_nm']].astype(str)
            result_df.loc[:,['id','dt_cnt','cust_cnt_all','dt_cnt_all']] = result_df[['id','dt_cnt','cust_cnt_all','dt_cnt_all']].astype(int)
            result_df.loc[:,['dt_cnt_rt_all','prob_wt','prob_wt_rank','lift','lift_rank','prob_wt_top10','lift_top10','lift_rank_top10']] = result_df[['dt_cnt_rt_all','prob_wt','prob_wt_rank','lift','lift_rank','prob_wt_top10','lift_top10','lift_rank_top10']].astype(float)
#             Upload_parquet_to_s3(result_df,filename)
            del result_df
            gc.collect()
        except Exception as e:
            print(f'Multiprocessing Error : {e}')
            p.close()
            p.terminate()
            time.sleep(2)
    time.sleep(10)

    print("          Multiprocessing for Logic_1 result is completed...", np.round((time.time() - start_time)/60,2),"mins")
    print(f"          Exceptional treatment needs to be applied to customer n = {len(result_list)} ...", np.round((time.time() - start_time)/60,2),"mins")

    ## Print progress 
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
          " || [Logic 1 : Prob Weight] calculation ",
          "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
    
    return 0 # i # result_list # res 


# Step 5) Logic2 - Micro-Segment 기준 장바구니 분석 
# ====================================================
# 2.5. 상품추천 로직 2 : Micro-segment
# ====================================================
## -----------------------------------------------
## 2.5.1. 함수 세팅 및 Micro-segment 후보군 도출 (Top1, Top2, Top3 토픽)
## -----------------------------------------------

def recommender_logic_2_micro_segment_creation(seg_n, Topic_df) :

    global target_date
    global model_target_nm

    """
    params 
    -----------------
    @ seg_n : 몇 개의 topic 으로 micro-segment 를 만들 것인지 설정
    @ Topic_df : micro_segment 생성을 위한 Topic_df input 
    @ df_raw : DT_CNT 집계를 위한 원본 데이터 input 

    return 
    -----------------
    1. seg_df : 고객 cust_id 별 최종 micro-segment 번호 

    """
    start_time = time.time()
    print("          Topic parsing is getting started..", np.round((time.time() - start_time)/60, 2),"mins")    

    ## Step 1 : seg_n 수에 맞춰 micro_segment : 1 ~ n 만큼 DataFrame 을 생성
    seg_list = []

    for i in range(seg_n) :
        print(f"          Creating Micro-segment using Top {i+1} topic is in progress..", np.round((time.time() - start_time)/60, 2),"mins")
        res = Topic_df.apply(recommender_logic_2_micro_segment_parsing, seg_count = i+1, axis = 1)
        res.name = 'seg_' + str(i+1) + 'grp'
        res = pd.DataFrame(res).set_index(Topic_df['intg_cust_no']).reset_index()
        seg_list.append(res)

    seg_df = reduce(lambda x,y: pd.merge(x,y, how = 'left',on = 'intg_cust_no'), seg_list)
    print("          Topic parsing is completed..", np.round((time.time() - start_time)/60, 2),"mins")


    seg_df['anl_dt'] = datetime.datetime.strptime(target_date, '%Y-%m-%d') 
#     seg_df['model_version'] = model_target_nm

    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
      " || Micro-segment creation is completed.",
      "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")

    return seg_df

def recommender_logic_2_micro_segment_parsing(a, seg_count) :
    """
    토픽확률이 0 이상인 토픽만 발라옴
    만약 토픽이 3개 미만인 고객의 경우 9999 처리
    """
    try : 
        a_flt = [i for i in a if type(i) == tuple and i[1] > 0 ]

        if len(a_flt) < seg_count : 
            res = [x[0] for x in a_flt] + [9999] * (seg_count - len(a_flt))
            a_sort = [str(i) for i in np.sort(res)]
            a_join = '@'.join(a_sort)
        else : 
            res = [x[0] for x in a_flt[:seg_count]]
            a_sort = [str(i) for i in np.sort(res)]
            a_join = '@'.join(a_sort)
    except : 
        a_sort = ['9999'] *3
        a_join = '@'.join(a_sort)

    return a_join


## Step 5-2) 고객xMicro-seg추출 및 Seg Size 계산
def make_cust_micro_seg_result(seg_result,seg_size):
    seg1_summary_df = seg_result.groupby(['seg_1grp'])['intg_cust_no'].count().reset_index()
    seg1_summary_df.columns = ['seg_1grp','seg_1grp_cnt']
    seg2_summary_df = seg_result.groupby(['seg_2grp'])['intg_cust_no'].count().reset_index()
    seg2_summary_df.columns = ['seg_2grp','seg_2grp_cnt']
    seg3_summary_df = seg_result.groupby(['seg_3grp'])['intg_cust_no'].count().reset_index()
    seg3_summary_df.columns = ['seg_3grp','seg_3grp_cnt']

    seg_result = pd.merge(seg_result,seg1_summary_df,how='left',on=['seg_1grp'])
    seg_result = pd.merge(seg_result,seg2_summary_df,how='left',on=['seg_2grp'])
    seg_result = pd.merge(seg_result,seg3_summary_df,how='left',on=['seg_3grp'])
    # seg size( number of micro seg intg cust_no) & seg final flag (last micro_seg)
    conditions = [(seg_result['seg_3grp_cnt'] > seg_size),
                  (seg_result['seg_2grp_cnt'] > seg_size),
                  (seg_result['seg_1grp_cnt'] > seg_size)]
    
    choices = [seg_result['seg_3grp'],seg_result['seg_2grp'],seg_result['seg_1grp']]
    choices2 = ['3','2','1']
    seg_result['seg_final'] = np.select(conditions, choices, default='9999@9999@9999')
    seg_result['seg_flag'] = np.select(conditions, choices2, default='9')
    # topic missing 
    seg_result['seg_3grp_missing'] = np.where(seg_result['seg_3grp'].str.contains('9999'),1,0)
    seg_result['seg_2grp_missing'] = np.where(seg_result['seg_2grp'].str.contains('9999'),1,0)
    seg_result['seg_1grp_missing'] = np.where(seg_result['seg_1grp'].str.contains('9999'),1,0)
    seg_result['mdl_ver'] = model_target_nm
    seg_result.columns = [i.upper() for i in seg_result.columns]
    return seg_result


def Delete_S3_File(filename):
    s3 = boto3.client("s3",
                          aws_access_key_id = AWS_ACCESS_KEY_ID,
                          aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                          region_name = AWS_DEFAULT_REGION)
    file_path =  f's3://{BUCKET_NAME}/{filename}'
    print(file_path)
    s3.delete_object(Bucket=BUCKET_NAME, Key=filename)   
    print(f'''============= Delete '{filename}' in S3 Bucket ==============''')



# Step1) LDA Model & APPLY 데이터 로드 및 Corpus생성(TF-IDF)

In [7]:
### definition of argument
start_time = time.time()

### Number of logic1 topic 
topN = 10

target_date_6digits = datetime.datetime.strptime(target_date, '%Y-%m-%d').strftime('%Y%m%d')
print(target_date, "target date : (",target_date_6digits,")"," operation  is getting started..")

model_path = MODEL_DIR

### parquet file load ###

df = rs_connect.query_request(f'''
                            select A.*
                             from LMDAPT3_DB.F_ANL_PROD_RECOM_APPLY A
                            where anl_dt = '{target_date}'
                            LIMIT 100000
                            ''')
# cl = dm.data_management("RECOM", target_date)
# df = cl.data_load()
df.columns = [i.upper() for i in df.columns]
df['DT_CNT'] = df['DT_CNT'].astype(int)
    

lda_model_iter,prdt_map,model_target_nm = LDA_Model_load(model_path)
print(f''' LDA Model Version : '{model_target_nm}' ''')


print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step1) Load Apply Data Complete",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
log_time = np.round((time.time() - start_time) / 60, 2)


prdt_df, corpus = data_preparation(df)
del df
gc.collect()

2022-11-01 target date : ( 20221101 )  operation  is getting started..
==== Load LDA Model Name : lda_model_221030.model ====
 LDA Model Version : 'lda_model_221030.model' 
2022-12-01 15:59:12  || Step1) Load Apply Data Complete (elapsed time : 0.02 mins)
          tf.shape : (56591, 1498) idf.shape : (1498,) tfidf.shape : (56591, 1498)
2022-12-01 15:59:42  || TF-IDF : Data Preprocessing is completed. (elapsed time : 0.5 mins)


0

In [9]:
corpus

INTG_CUST_NO
00000000000654                  [(889, 9.0)]
00000000000935       [(40, 7.0), (826, 3.0)]
00000000001504    [(1070, 1.0), (2164, 3.0)]
00000000001644    [(951, 27.0), (1087, 1.0)]
00000000002195                  [(355, 3.0)]
                             ...            
00000046152706                  [(142, 5.0)]
00000046152747                  [(435, 1.0)]
00000046159655                  [(831, 1.0)]
00000046159957                 [(1518, 1.0)]
00000046160471                 [(2410, 1.0)]
Length: 56591, dtype: object

# Step2) Logic1 - 가중확률곱 & Lift 계산

In [10]:
## Logic1 - Cust*Topic distribution
Topic_df, Topic_df_to_sql = Topic_df_converter(lda_model = lda_model_iter, corpus = corpus)
Topic_df_TopN, topic_distribution = topic_distribution(n=3,lda_model=lda_model_iter,Topic_df=Topic_df, nword = 30)

## Filtering remove cust_id
bf_n = len(corpus)
cust_id_remove = corpus[corpus.map(lambda x: len(x)) < 1].index.tolist()

corpus_iter = corpus[~corpus.index.isin(cust_id_remove)]
Topic_df = Topic_df[~Topic_df.intg_cust_no.isin(cust_id_remove)]

## Calculate Logic1 probwt

### Schema_inspector (drop & create)

# ## ------------------------------------------------
# ## 2.4.2. 최종 적재용 테이블 생성용 데이터타입 지정
# ## ------------------------------------------------

# col_dtype_for_query_final
schema   = 'lmdapwk_db'
tbl_name = f't_anl_logic1_probwt_result'

# rs_connect.cud_request(f'''DELETE FROM {schema}.{tbl_name}''')
# print(f'''DELETE FROM {schema}.{tbl_name}is success''')
# logger_success.error(f'''*** DELETE FROM {schema}.{tbl_name}is success''')

##### Multi-Processing
warnings.filterwarnings('ignore')

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step2) Logic1 probwt calculate getting started.. ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
log_time = np.round((time.time() - start_time) / 60, 2)
print(f'''*** (Step2) -  Logic1 probwt calculate getting started.. | (elapsed time : {log_time} mins)''')

### load l3 prod dt rate ###
basket_agg_df = Select_Logic1_baske_df(target_date)
basket_agg_df.columns = [x.lower() for x in basket_agg_df.columns]

input_list = range(len(corpus_iter))
print(f''' Corpus_iter length : {len(input_list)}''')
print(f'''*** Corpus_iter length : {len(input_list)}''')

filename = 't_anl_logic1_probwt_result'
file_path = f's3://{BUCKET_NAME}/{filename}'

if len(corpus_iter) > 400000 : 
    res_df1 = []
    input_list_split = np.array_split(input_list, 2)
    for k, l_iter in enumerate(input_list_split) :
        print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
         f" || [Logic 1 : Prob Weight] {k+1}/2 of multiprocessing is getting started.." )

        print(f'*** [Logic 1 : Prob Weight] {k+1}/2 of multiprocessing is getting started.."')

        res_df1 += multiprocesser_logic1_tqdm(worker = recommender_logic_1,  arg_list = l_iter, n_core = 5)
else : 
    res_df1 = multiprocesser_logic1_tqdm(worker = recommender_logic_1,  arg_list = input_list, n_core = 10,filename = filename)

# create engine(sqlalchemy -> copy s3 to redshift)  
# engine = create_engine(f'redshift+psycopg2://{USER_ID}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')

# engine.execute(f"COPY {schema}.{tbl_name} FROM '{file_path}' iam_role '{IAM_ARN}' \
# #               FORMAT AS PARQUET; commit;")
print('================ Copy file t_anl_logic1_probwt_result S3 to Redshift was successfully  ==================')
print('*** Copy file t_anl_logic1_probwt_result S3 to Redshift was successfully')

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step2) Logic1 probwt calculate complete.. ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
log_time = np.round((time.time() - start_time) / 60, 2)
print(f'''*** (Step2) -  Logic1 probwt calculate complete.. | (elapsed time : {log_time} mins)''')


          Topic_df_converter : Step 1-1. Loading topic is completed.. 0.0 mins
          Topic_df_converter : Step 1-2. Sparse Matrix Coversion is completed.. 0.38 mins
          Topic_df_converter : Step 1-3. Sparse Matrix to Array Transformation is completed.. 0.38 mins
          Topic_df_converter : Step 1-4. Gensim object --> DF conversion is completed.. 0.38 mins
          Topic_df_converter : Step 1-5. Tuple & dimension changes are completed.. 0.41 mins
          Topic_df_converter : Step 1-6. DF transformation is completed.. 0.45 mins
2022-12-01 16:01:19  || Topic DF generation is completed. (elapsed time : 0.45 mins)
The No. of Customers who has topic : 56591 / 56591
The No. of Customers who don't have topic : 0 / 56591
2022-12-01 16:01:20  || Step2) Logic1 probwt calculate getting started..  (elapsed time : 2.16 mins)
*** (Step2) -  Logic1 probwt calculate getting started.. | (elapsed time : 2.16 mins)
 Corpus_iter length : 56591
*** Corpus_iter length : 56591
          Multip

100%|████████████████████████████████████| 56591/56591 [04:08<00:00, 228.10it/s]


1
2
3
4
          Multiprocessing for Logic_1 result is completed... 6.21 mins
          Exceptional treatment needs to be applied to customer n = 56591 ... 6.21 mins
2022-12-01 16:07:45  || [Logic 1 : Prob Weight] calculation  (elapsed time : 6.21 mins)
================ Copy file t_anl_logic1_probwt_result S3 to Redshift was successfully  ==================
*** Copy file t_anl_logic1_probwt_result S3 to Redshift was successfully
2022-12-01 16:07:46  || Step2) Logic1 probwt calculate complete..  (elapsed time : 8.59 mins)
*** (Step2) -  Logic1 probwt calculate complete.. | (elapsed time : 8.59 mins)


# Step3) Logic2 - Micro-seg생성(TOP3 토픽)

In [14]:
## Logic2 - Micro-Seg basket agg

insert_date_from = (datetime.datetime.strptime(target_date, '%Y-%m-%d') - timedelta(days = 6)).strftime('%Y-%m-%d')
insert_date_to = (datetime.datetime.strptime(target_date, '%Y-%m-%d') - timedelta(days = int(1))).strftime('%Y-%m-%d')

date_list = pd.date_range(start=insert_date_from,end=insert_date_to).strftime('%Y-%m-%d').tolist()

schema    = 'lmdapwk_db'
tbl_name  = 'T_ANL_PROD_RECOM_CUST_MICRO_SEG'

micro_seg_chk_sql = f'''
          SELECT *
          FROM {schema}.{tbl_name}
          WHERE ANL_DT BETWEEN '{insert_date_from}' AND '{insert_date_to}'
          AND MDL_VER = '{model_target_nm}'
          ORDER BY ANL_DT
          '''

micro_seg_cust_df = rs_connect.query_request(micro_seg_chk_sql)
micro_seg_cust_df['ANL_DT'] = micro_seg_cust_df['ANL_DT'].apply(lambda x: x.strftime('%Y-%m-%d'))

new_ms_date = list(set(date_list)-set(micro_seg_cust_df.ANL_DT.unique().tolist()))

### 모델버전 변경시 분석기준일치의(1차모델링시 7일) Micro-seg 재생성

if len(new_ms_date) > 0:
    seg_result  = pd.DataFrame()
    for t in new_ms_date: 
        try:
            print(f'''Date : {t}''' )
            sql = f'''SELECT * FROM LMDAPT3_DB.F_ANL_PROD_RECOM_APPLY WHERE ANL_DT = '{t}' '''
            data_set = rs_connect.query_request(sql)

            df_flt = data_set[['INTG_CUST_NO','ANL_DT','L3_CD','DT_CNT']]
            prdt_df, corpus2 = data_preparation(df_flt)

#             micro_seg_corpus = Load_corpus(filename)
            Topic_df2, Topic_df_to_sql = Topic_df_converter(lda_model = lda_model_iter, corpus = corpus2)

         # seg_df 생성하여 DF저장
            seg_df_r = recommender_logic_2_micro_segment_creation(3, Topic_df2)

            seg_df_r['anl_dt'] = t
            seg_result = pd.concat([seg_result,seg_df_r])
        except Exception as e:
            print(e)
    seg_result['cust_rank'] = seg_result.groupby(['intg_cust_no'])['anl_dt'].rank(ascending=False)
    seg_df = seg_result[seg_result['cust_rank']==1]
    micro_seg = make_cust_micro_seg_result(seg_df,4)
    micro_seg['LOAD_DT'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    micro_seg_cols = ['MDL_VER','ANL_DT','INTG_CUST_NO', 'SEG_1GRP', 'SEG_2GRP', 'SEG_3GRP','SEG_1GRP_CNT', 'SEG_2GRP_CNT', 'SEG_3GRP_CNT','SEG_FINAL', 'SEG_FLAG' ,'LOAD_DT']
#     rs_connect.replace_table(micro_seg[micro_seg_cols],'lmdapwk_db.T_ANL_PROD_RECOM_CUST_MICRO_SEG',True)
else:
    print('6일치 micro-seg데이터가 있습니다.')
    seg_df = recommender_logic_2_micro_segment_creation(3, Topic_df)

    micro_seg = make_cust_micro_seg_result(seg_df,4)
    micro_seg['LOAD_DT'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    micro_seg_cols = ['MDL_VER','ANL_DT','INTG_CUST_NO', 'SEG_1GRP', 'SEG_2GRP', 'SEG_3GRP','SEG_1GRP_CNT', 'SEG_2GRP_CNT', 'SEG_3GRP_CNT','SEG_FINAL', 'SEG_FLAG','LOAD_DT']
#     rs_connect.cud_request(f''' DELETE FROM lmdapwk_db.T_ANL_PROD_RECOM_CUST_MICRO_SEG WHERE ANL_DT = '{target_date}' ''')
#     rs_connect.replace_table(micro_seg[micro_seg_cols],'lmdapwk_db.T_ANL_PROD_RECOM_CUST_MICRO_SEG',True)
    del micro_seg,seg_df
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step3) Logic2 Cust Micro-seg creation complete.. ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
    log_time = np.round((time.time() - start_time) / 60, 2)
    print(f'''*** (Step3) -  Logic2 Cust Micro-seg creation complete.. | (elapsed time : {log_time} mins)''')

6일치 micro-seg데이터가 있습니다.
          Topic parsing is getting started.. 0.0 mins
          Creating Micro-segment using Top 1 topic is in progress.. 0.0 mins
          Creating Micro-segment using Top 2 topic is in progress.. 0.05 mins
          Creating Micro-segment using Top 3 topic is in progress.. 0.1 mins
          Topic parsing is completed.. 0.16 mins
2022-12-01 16:30:36  || Micro-segment creation is completed. (elapsed time : 0.16 mins)
2022-12-01 16:30:37  || Step3) Logic2 Cust Micro-seg creation complete..  (elapsed time : 31.43 mins)
*** (Step3) -  Logic2 Cust Micro-seg creation complete.. | (elapsed time : 31.43 mins)


# Step4) Logic2 - Micro-seg별 90일 장바구니 추출

In [ ]:
logic2_sql = f'''
WITH CUST_RECMD_MICROSEG_BASE_RAW as (SELECT T1.intg_cust_no,T1.l3_cd ,T1.l3_nm ,T1.anl_dt ,T1.dt_cnt
                                           ,T2.SEG_1GRP ,T2.SEG_2GRP ,T2.SEG_3GRP,T2.SEG_1GRP_CNT ,T2.SEG_2GRP_CNT ,T2.SEG_3GRP_CNT ,T2.SEG_FINAL ,T2.seg_final 
                                     FROM  LMDAPT3_DB.F_ANL_PROD_RECOM_APPLY T1
                                     INNER join lmdapwk_db.T_ANL_PROD_RECOM_CUST_MICRO_SEG T2 on T1.intg_cust_no = T2.INTG_CUST_NO and T1.anl_dt = T2.ANL_DT  
                                     WHERE T1.ANL_DT BETWEEN TO_DATE(dateadd(day,-6,'{target_date}'),'YYYY-MM-DD') AND DATE('{target_date}'))
,TMP AS (
          SELECT
               SEG_3GRP AS SEG_FINAL
              ,L3_CD,L3_NM
              ,SUM(DT_CNT) AS DT_CNT
              ,MAX(SEG_3GRP_CNT) AS SEG_CNT
              ,cast(SUM(DT_CNT)as decimal) / cast(MAX(SEG_3GRP_CNT)as decimal) AS DT_CNT_RT 
          FROM CUST_RECMD_MICROSEG_BASE_RAW
          GROUP BY SEG_3GRP, L3_CD,L3_NM
      UNION ALL
          SELECT 
               SEG_2GRP AS SEG_FINAL
              ,L3_CD,L3_NM 
              ,SUM(DT_CNT) AS DT_CNT
              ,MAX(SEG_2GRP_CNT) AS SEG_CNT
              ,cast(SUM(DT_CNT)as decimal) / cast(MAX(SEG_2GRP_CNT) as DECIMAL) AS DT_CNT_RT 
          FROM CUST_RECMD_MICROSEG_BASE_RAW
          GROUP BY SEG_2GRP, L3_CD,L3_NM
      UNION ALL
          SELECT
               SEG_1GRP AS SEG_FINAL
              ,L3_CD,L3_NM
              ,SUM(DT_CNT) AS DT_CNT
              ,MAX(SEG_1GRP_CNT) AS SEG_CNT
              ,cast(SUM(DT_CNT)as decimal) / cast(MAX(SEG_1GRP_CNT)as decimal) AS DT_CNT_RT 
          FROM CUST_RECMD_MICROSEG_BASE_RAW
          GROUP BY SEG_1GRP, L3_CD,L3_NM
      UNION ALL
          SELECT 
              'ALL' AS SEG_FINAL
              ,A.L3_CD,A.L3_NM
              ,SUM(A.DT_CNT) AS DT_CNT
              ,MAX(B.ALL_CNT) AS SEG_CNT
              ,cast(SUM(DT_CNT)as decimal) / cast(MAX(B.ALL_CNT)as decimal) AS DT_CNT_RT
          FROM CUST_RECMD_MICROSEG_BASE_RAW A,
               (SELECT COUNT(DISTINCT INTG_CUST_NO) AS ALL_CNT FROM CUST_RECMD_MICROSEG_BASE_RAW) B 
          GROUP BY A.L3_CD,A.L3_NM
      ),
    TMP2 AS (
          SELECT COUNT(DISTINCT INTG_CUST_NO) AS ALL_CNT FROM CUST_RECMD_MICROSEG_BASE_RAW
      )

      SELECT  '{target_date}' as ANL_DT
              ,A.SEG_FINAL
              ,A.L3_CD , A.L3_NM 
              ,A.DT_CNT_BASKET
              ,A.SEG_CNT
              ,A.DT_CNT_RT_BASKET
              ,A.BASKET_RANK
              ,B.DT_CNT_ALL
              ,B.CUST_CNT_ALL
              ,B.DT_CNT_RT_ALL
              ,A.DT_CNT_RT_BASKET * (SELECT ALL_CNT FROM TMP2) / B.DT_CNT_ALL AS LIFT
              ,DENSE_RANK() OVER (PARTITION BY SEG_FINAL ORDER BY A.DT_CNT_RT_BASKET * (SELECT ALL_CNT FROM TMP2) / B.DT_CNT_ALL DESC ) AS LIFT_RANK

              /* MICROSEG 보삽 기준 : 
                 MICROSEG 고객 수 기준 4분위수 이상인 경우 (= 20)

                 -->  DT_CNT_BASKET (품목별 구매일수-DT_CNT) < 4 미만인 경우 0 처리 */

              ,CASE WHEN A.SEG_CNT > 20 AND A.DT_CNT_BASKET < 4 THEN 0 
                    ELSE A.DT_CNT_RT_BASKET * (SELECT ALL_CNT FROM TMP2) / B.DT_CNT_ALL END AS LIFT_FLT_OVER_Q4
              ,DENSE_RANK() OVER (PARTITION BY SEG_FINAL ORDER BY 
                                  (CASE WHEN A.SEG_CNT > 20 AND A.DT_CNT_BASKET < 4 THEN 0 
                                        ELSE A.DT_CNT_RT_BASKET * (SELECT ALL_CNT FROM TMP2) / B.DT_CNT_ALL END) DESC) AS LIFT_RANK_FLT_OVER_Q4

      FROM (
          SELECT  SEG_FINAL
                 ,L3_CD ,L3_NM 
                 ,DT_CNT AS DT_CNT_BASKET
                 ,SEG_CNT
                 ,DT_CNT_RT AS DT_CNT_RT_BASKET
                 ,DENSE_RANK() OVER (PARTITION BY SEG_FINAL ORDER BY DT_CNT_RT DESC) AS BASKET_RANK
           FROM TMP
           WHERE DT_CNT > 1 -- MICROSEG 내 구매 이력 중 구매이력이 1 인 소분류 는 제외
           ORDER BY SEG_FINAL,L3_CD ,L3_NM ASC ) A

      -- 전체 장바구니 구매 내역을 붙여 LIFT 계산
      LEFT JOIN (
          SELECT  L3_CD ,L3_NM 
                  ,DT_CNT AS DT_CNT_ALL
                  ,SEG_CNT AS CUST_CNT_ALL
                  ,DT_CNT_RT AS DT_CNT_RT_ALL
          FROM  TMP
          WHERE  SEG_FINAL = 'ALL'
      ) B ON A.L3_CD = B.L3_CD 
 '''
logic2_df = rs_connect.query_request(logic2_sql)
inttype_cols = ['DT_CNT_BASKET', 'SEG_CNT', 'DT_CNT_ALL', 'CUST_CNT_ALL']

logic2_df[inttype_cols] = logic2_df[inttype_cols].astype(int)

# rs_connect.cud_request(f''' DELETE FROM lmdapwk_db.t_anl_logic2_microseg_basket ''')
# rs_connect.replace_table(logic2_df,'lmdapwk_db.t_anl_logic2_microseg_basket',True)

del logic2_df
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step4) Logic2 Cust Micro-seg basket calculate SQL complete.. ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
log_time = np.round((time.time() - start_time) / 60, 2)


# Step 5) Logic2 - Micro-seg Lift스코어 계산

In [ ]:
logic2_result_sql = f'''
                  SELECT  LA.ANL_DT                   -- 분석 일자
                          ,LA.INTG_CUST_NO            -- 고객 번호
                          ,LA.SEG_1GRP                -- TOP 1 TOPIC 으로 만든 MICRO-SEG
                          ,LA.SEG_2GRP                -- TOP 2 TOPICS 로 만든 MICRO-SEG
                          ,LA.SEG_3GRP                -- TOP 3 TOPICS 로 만든 MICRO-SEG
                          ,LA.SEG_1GRP_CNT            -- SEG_1GRP 에 해당하는 고객 수
                          ,LA.SEG_2GRP_CNT            -- SEG_2GRP 에 해당하는 고객 수
                          ,LA.SEG_3GRP_CNT            -- SEG_3GRP 에 해당하는 고객 수
                          ,LA.SEG_FINAL               -- 고객 별 최종 MICRO-SEG
                          ,LA.SEG_FLAG                -- MICRO-SEG 종류에 해당하는 FLAG (3: SEG_3GRP / 2: SEG_2GRP / 1: SEG_1GRP)
                          ,COALESCE(LA.L3_CD, LB.L3_CD) AS L3_CD       -- 상품 소분류 코드
                          ,COALESCE(LA.L3_NM, LB.L3_NM) AS L3_NM       -- 상품 소분류 이름
                          ,LB.DT_CNT                  -- 해당 고객 별 상품 구매 횟수 (1일 MAX 1회)
                          ,LA.DT_CNT_BASKET           -- MICRO-SEG 별 상품 구매 횟수
                          ,LA.SEG_CNT                 -- 해당 MICRO-SEG 의 고객 수 (SEG_1/2/3_GRP_CNT 와 동일)
                          ,LA.DT_CNT_RT_BASKET        -- MICRO-SEG 장바구니 비중 ( = DT_CNT_BASKET / SEG_CNT 로 산출된 비율)
                          ,LA.BASKET_RANK             -- MICRO-SEG 장바구니 비중으로 산출된 랭킹 (내림차순)
                          ,LA.DT_CNT_ALL              -- 전체 고객의 상품 구매 횟수
                          ,LA.CUST_CNT_ALL            -- 최근7일간 고객 총 수 (= COUNT(DISTINCT INTG_CUST_NO))
                          ,LA.DT_CNT_RT_ALL           -- 전체 고객의 장바구니 비중
                          ,LA.LIFT                    -- 전체 고객의 장바구니 비중 대비 MICRO-SEG 장바구니 비중 LIFT
                          ,LA.LIFT_RANK               -- LIFT 로 산출된 랭킹 (내림차순)
                          ,LA.LIFT_FLT_OVER_Q4        -- [LIFT 보정] SEG_SIZE >= 50 이상 MICRO-SEG 대상 구매건수 5 미만 품목 제외한 LIFT 값
                          ,LA.LIFT_RANK_FLT_OVER_Q4   -- [LIFT 보정] LIFT_FLT_OVER_Q4 기준 LIFT_RANK
                  FROM (
                      SELECT  SA.ANL_DT ,SA.INTG_CUST_NO ,SA.SEG_1GRP ,SA.SEG_2GRP ,SA.SEG_3GRP
                             ,SA.SEG_1GRP_CNT ,SA.SEG_2GRP_CNT ,SA.SEG_3GRP_CNT
                             ,SA.SEG_FINAL ,SA.SEG_FLAG
                             ,SB.L3_CD ,SB.L3_NM 
                             ,SB.DT_CNT_BASKET
                             ,SB.SEG_CNT ,SB.DT_CNT_RT_BASKET ,SB.BASKET_RANK
                             ,SB.DT_CNT_ALL ,SB.CUST_CNT_ALL ,SB.DT_CNT_RT_ALL
                             ,SB.LIFT ,SB.LIFT_RANK
                             ,SB.LIFT_FLT_OVER_Q4, SB.LIFT_RANK_FLT_OVER_Q4

                        -- TARGET_DATE 에 맞춰 해당 날짜의 INTG_CUST_NO 및 MICROSEG ID 를 끌고 옴
                        FROM (SELECT ANL_DT ,A.INTG_CUST_NO ,SEG_1GRP ,SEG_2GRP ,SEG_3GRP ,SEG_1GRP_CNT ,SEG_2GRP_CNT ,SEG_3GRP_CNT 
                                    ,SEG_FINAL ,SEG_FLAG
                                FROM lmdapwk_db.T_ANL_PROD_RECOM_CUST_MICRO_SEG  A
                               WHERE ANL_DT = '{target_date}' 
                             ) SA
                        LEFT JOIN lmdapwk_db.T_ANL_LOGIC2_MICROSEG_BASKET SB
                               ON SA.SEG_FINAL = SB.SEG_FINAL 
                  ) LA
                  -- 고객의 구매/미구매이력과 상품추천 결과와의 비교를 위해 가져옴
                  LEFT JOIN (
                         SELECT INTG_CUST_NO, ANL_DT, L3_CD, L3_NM, DT_CNT 
                           FROM LMDAPT3_DB.F_ANL_PROD_RECOM_APPLY
                          WHERE ANL_DT = '{target_date}' ) LB 
                    ON  LA.INTG_CUST_NO = LB.INTG_CUST_NO 
                          AND LA.L3_CD  = LB.L3_CD
                          AND LA.ANL_DT = LB.ANL_DT 
                  WHERE (LA.LIFT_RANK <= 100) 
                        OR (LA.BASKET_RANK <= 100) 
                        OR (LA.LIFT_RANK_FLT_OVER_Q4 <= 100)
                        OR (LB.DT_CNT IS NOT NULL)
                  '''

logic2_result_df =  rs_connect.query_request(logic2_result_sql)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step5) Logic2 Lift score calculate SQL complete.. ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
log_time = np.round((time.time() - start_time) / 60, 2)


## Copy S3 to redshift (LOGIC2 CUST MICRO-SEG RESULT TABLE) : 4m 32s

str_cols = ['ANL_DT','INTG_CUST_NO','SEG_1GRP','SEG_2GRP','SEG_3GRP','SEG_FINAL','SEG_FLAG','L3_CD','L3_NM']
int_cols = ['SEG_1GRP_CNT','SEG_2GRP_CNT','SEG_3GRP_CNT','DT_CNT_BASKET','SEG_CNT','BASKET_RANK','DT_CNT_ALL','CUST_CNT_ALL','LIFT_RANK','LIFT_RANK_FLT_OVER_Q4']
float_cols = ['DT_CNT','DT_CNT_RT_BASKET','DT_CNT_RT_ALL','LIFT','LIFT_FLT_OVER_Q4']

logic2_result_df.loc[:,str_cols] = logic2_result_df.loc[:,str_cols].astype(str)
logic2_result_df[int_cols]       = logic2_result_df[int_cols].astype(int)
logic2_result_df[float_cols]     = logic2_result_df[float_cols].astype(float)

# Upload_parquet_to_s3(logic2_result_df,'t_anl_logic2_microseg_result')

# col_dtype_for_query_final
schema = 'lmdapwk_db'
tbl_name = f't_anl_logic2_microseg_result'
filename = 't_anl_logic2_microseg_result'
file_path = f's3://{BUCKET_NAME}/{filename}'


#     Schema_inspector(schema,tbl_name) 
# rs_connect.cud_request(f'''DELETE FROM {schema}.{tbl_name} ''')
# print(f'''DELETE FROM {schema}.{tbl_name} is success''')
# logger_success.error(f'''*** DELETE FROM {schema}.{tbl_name} is success''')

# # create engine(sqlalchemy -> copy s3 to redshift)  
# engine = create_engine(f'redshift+psycopg2://{USER_ID}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')

# engine.execute(f"COPY {schema}.{tbl_name} FROM '{file_path}' iam_role '{IAM_ARN}' \
#               FORMAT AS PARQUET; commit;")
print('================ Copy file S3 t_anl_logic2_microseg_result to Redshift was successfully  ==================')
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step5) Logic2 Lift score result db upload complete.. ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")


# Step 6) Logic1추천결과 + Logic2 추천결과 결합(Full outer join)

In [ ]:
##  최종테이블 생성 : Logic1+Logic2 (13m 22s)
combine_sql = f'''
          SELECT    COALESCE(D.ANL_DT,C.ANL_DT) AS ANL_DT
                   ,COALESCE(D.INTG_CUST_NO, C.INTG_CUST_NO) AS INTG_CUST_NO
                   ,COALESCE(D.L3_CD, C.L3_CD) AS L3_CD
                   ,COALESCE(D.L3_NM, C.L3_NM) AS L3_NM
                  ,D.DT_CNT
                  /* -- 미구매상품 (CROSS-SELL 대상 상품) / 기구매상품 (RE-SELL 대상 상품) 2가지로만 구분 -- */

                  ,CASE WHEN (D.DT_CNT IS NULL) OR (D.DT_CNT = 0) THEN 'CROSS-SELL'
                        WHEN (D.DT_CNT IS NOT NULL) OR (D.DT_CNT > 0) THEN 'RE-SELL' END AS SELL_TYPE
                  /* Logic1 확률곱, 확률곱 순위 */
                  ,C.PROB_WT, C.PROB_WT_RANK
                  ,C.LIFT_TOP10 AS LIFT_TOP10_TOPIC, C.LIFT_RANK_TOP10 AS LIFT_RANK_TOP10_TOPIC

                  ,D.DT_CNT_RT_BASKET AS DT_CNT_RT_MICROSEG_BASKET, D.BASKET_RANK AS RANK_MICROSEG_BASKET 
                  ,D.LIFT_FLT_OVER_Q4 AS LIFT_MICROSEG_BASKET, D.LIFT_RANK_FLT_OVER_Q4 AS LIFT_RANK_MICROSEG_BASKET

          FROM (
              SELECT  ANL_DT, INTG_CUST_NO  , L3_CD, L3_NM, DT_CNT
                     ,PROB_WT, PROB_WT_RANK
                     ,LIFT_TOP10, LIFT_RANK_TOP10
                FROM  lmdapwk_db.T_ANL_LOGIC1_PROBWT_RESULT   ) C

          FULL OUTER JOIN (
              SELECT   ANL_DT, INTG_CUST_NO, SEG_FINAL, L3_CD, L3_NM, DT_CNT
                      ,DT_CNT_RT_BASKET, BASKET_RANK
                      ,LIFT_FLT_OVER_Q4, LIFT_RANK_FLT_OVER_Q4

                FROM  lmdapwk_db.T_ANL_LOGIC2_MICROSEG_RESULT ) D 
              ON  C.INTG_CUST_NO = D.INTG_CUST_NO 
                  AND CAST(C.L3_CD AS VARCHAR) = CAST(D.L3_CD AS VARCHAR);

      '''
temp_df = rs_connect.query_request(combine_sql)

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step6) Logic1+Logic2 table combine SQL complete.. ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")


# Upload_parquet_to_s3(temp_df,'t_anl_prod_recmd_combine_result')
del temp_df

# col_dtype_for_query_final
schema    = 'lmdapwk_db'
tbl_name  = f't_anl_prod_recmd_combine_result'
filename  = f't_anl_prod_recmd_combine_result'
file_path = f's3://{BUCKET_NAME}/{filename}'

# rs_connect.cud_request(f'''DELETE FROM {schema}.{tbl_name} ''')
# print(f'''DELETE FROM {schema}.{tbl_name} is success''')
# logger_success.error(f'''*** DELETE FROM {schema}.{tbl_name} is success''')

## create engine(sqlalchemy -> copy s3 to redshift)  
# engine = create_engine(f'redshift+psycopg2://{USER_ID}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')

# engine.execute(f"COPY {schema}.{tbl_name} FROM '{file_path}' iam_role '{IAM_ARN}' \
#               FORMAT AS PARQUET; commit;")
print('================ Copy file S3 t_anl_prod_recmd_combine_result to Redshift was successfully  ==================')


print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step6) Logic1+Logic2 table DB Upload complete.. ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
log_time = np.round((time.time() - start_time) / 60, 2)

# Step7) Cross-sell / Re-sell MRR Score계산 

In [ ]:
mrr_sql = '''  
      WITH TMP AS (
              -- 랭킹 빈칸 보삽 진행 (일괄적으로 랭킹 9999 위로 지정)
              SELECT  ANL_DT, INTG_CUST_NO, L3_CD, L3_NM, DT_CNT, SELL_TYPE
                      ,PROB_WT
                      ,COALESCE(PROB_WT_RANK, 9999) AS PROB_WT_RANK
                      ,LIFT_TOP10_TOPIC
                      ,(case when LIFT_RANK_TOP10_TOPIC is null or LIFT_RANK_TOP10_TOPIC = 0 then 9999
                             else LIFT_RANK_TOP10_TOPIC end)  AS LIFT_RANK_TOP10_TOPIC
                      ,DT_CNT_RT_MICROSEG_BASKET
                      ,COALESCE(RANK_MICROSEG_BASKET,9999) AS RANK_MICROSEG_BASKET
                      ,LIFT_MICROSEG_BASKET
                      ,COALESCE(LIFT_RANK_MICROSEG_BASKET, 9999) AS LIFT_RANK_MICROSEG_BASKET
              FROM  lmdapwk_db.t_anl_prod_recmd_combine_result 
          ), 
          TMP2 AS (
              -- MRR 스코어와 RE-SELL/CR0SS-SELL FLAG 지정
              SELECT  ANL_DT, INTG_CUST_NO, L3_CD, L3_NM, DT_CNT, SELL_TYPE
                      ,PROB_WT
                      ,PROB_WT_RANK
                      ,LIFT_TOP10_TOPIC
                      ,LIFT_RANK_TOP10_TOPIC
                      ,DT_CNT_RT_MICROSEG_BASKET
                      ,RANK_MICROSEG_BASKET
                      ,LIFT_MICROSEG_BASKET
                      ,LIFT_RANK_MICROSEG_BASKET
                      -- MRR 계산
                      ,0.5 * ( 1/PROB_WT_RANK + 1/RANK_MICROSEG_BASKET )               AS RS_MRR -- RE-SELL 랭킹 스코어
                      ,0.5 * ( 1/LIFT_RANK_TOP10_TOPIC + 1/LIFT_RANK_MICROSEG_BASKET ) AS CS_MRR -- CROSS-SELL 랭킹 스코어
                      -- MRR 랭킹 계산용 FLAG
                      ,CASE WHEN SELL_TYPE = 'RE-SELL' THEN 1 ELSE 0 END AS RS_FLAG
                      ,CASE WHEN SELL_TYPE = 'CROSS-SELL' THEN 1 ELSE 0 END AS CS_FLAG

              FROM TMP
          ), 
          TMP3 AS (
              -- RE-SELL / CROSS-SELL 랭킹 계산
              SELECT  ANL_DT, INTG_CUST_NO, L3_CD, L3_NM, DT_CNT, SELL_TYPE
                      ,PROB_WT
                      ,PROB_WT_RANK
                      ,LIFT_TOP10_TOPIC
                      ,LIFT_RANK_TOP10_TOPIC
                      ,DT_CNT_RT_MICROSEG_BASKET
                      ,RANK_MICROSEG_BASKET
                      ,LIFT_MICROSEG_BASKET
                      ,LIFT_RANK_MICROSEG_BASKET
                      ,RS_MRR  -- RE-SELL 용 MRR 스코어
                      ,RS_FLAG -- RE-SELL 용 FLAG
                      ,CS_MRR  -- CROSS-SELL 용 MRR 스코어
                      ,CS_FLAG -- CROSS-SELL 용 FLAG
              FROM TMP2 
          )

          -- UNION 방식으로 TABLE APPEND 
      select T1.*,T2.l1_cd,T3.L1_NM,T4.HYPER_TEAM_CD,T4.HYPER_TEAM_NM,
             RANK() OVER(partition by INTG_CUST_NO ,SELL_TYPE ,T2.HYPER_TEAM_CD order by L3_RANK ASC,DT_CNT_RT_MICROSEG_BASKET DESC) as L1_RANK
      from (
              SELECT  ANL_DT, INTG_CUST_NO, A.L3_CD, A.L3_NM, DT_CNT, SELL_TYPE
                          ,PROB_WT
                          ,PROB_WT_RANK
                          ,LIFT_TOP10_TOPIC
                          ,LIFT_RANK_TOP10_TOPIC
                          ,DT_CNT_RT_MICROSEG_BASKET
                          ,RANK_MICROSEG_BASKET
                          ,LIFT_MICROSEG_BASKET
                          ,LIFT_RANK_MICROSEG_BASKET
                          ,RS_MRR AS MRR       
                          ,DENSE_RANK() OVER (PARTITION BY ANL_DT, INTG_CUST_NO ORDER BY RS_MRR DESC,PROB_WT DESC,LIFT_RANK_TOP10_TOPIC ASC ) as L3_RANK
              FROM   TMP3 A 
              WHERE  SELL_TYPE = 'RE-SELL' 
              AND ANL_DT IS NOT NULL
              UNION ALL (
                  SELECT  ANL_DT, INTG_CUST_NO, A.L3_CD, A.L3_NM, DT_CNT, SELL_TYPE
                          ,PROB_WT
                          ,PROB_WT_RANK
                          ,LIFT_TOP10_TOPIC
                          ,LIFT_RANK_TOP10_TOPIC
                          ,DT_CNT_RT_MICROSEG_BASKET
                          ,RANK_MICROSEG_BASKET
                          ,LIFT_MICROSEG_BASKET
                          ,LIFT_RANK_MICROSEG_BASKET
                          ,CS_MRR AS MRR       
                          ,DENSE_RANK() OVER (PARTITION BY ANL_DT, INTG_CUST_NO ORDER BY CS_MRR DESC,DT_CNT_RT_MICROSEG_BASKET DESC,LIFT_RANK_MICROSEG_BASKET ASC ) as L3_RANK
                  FROM   TMP3 A 
                  WHERE  SELL_TYPE = 'CROSS-SELL' 
                  AND ANL_DT IS NOT NULL
              )
      )T1
      left join lmdapt3_db.d_prod_l3 T2         ON T1.L3_CD = T2.l3_cd 
      left join lmdapt3_db.d_prod_l1 T3         ON T2.L1_CD = T3.l1_cd 
      left join lmdapt3_db.d_hyper_prod_team T4 ON T4.hyper_team_cd = T3.hyper_team_cd
      order by INTG_CUST_NO ,SELL_TYPE ,T1.L3_RANK,L1_CD,L1_RANK
'''
last_df = rs_connect.query_request(mrr_sql)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step7) Cross-sell / Re-sell MRR Score calculate complete.. ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
log_time = np.round((time.time() - start_time) / 60, 2)

# Step8) 10대 Or Demo정보 없는 고객 주류팀 상품제외,동일 대분류 기준 4위까지 추천되게 filtering

In [ ]:
cust_demo_df = rs_connect.query_request(f'''SELECT A.*,B.AGE_GRP_5_NM
                          FROM(SELECT 
                                '{target_date}' as ANL_DT
                               , T1.INTG_CUST_NO                                                                                                        /* 통합고객번호 */
                               , CASE WHEN (T3.AGE >= 150 OR T3.AGE <= 10) THEN 999
                                      ELSE T3.AGE
                                 END AS AGE                                                                                                             /* 연령 */
                               , CASE WHEN (T3.AGE >= 150 OR T3.AGE <= 10) THEN '99'
                                      ELSE T3.AGE_GRP_2FG_CD
                                 END AS AGE_GRP                                                                                                         /* 5세 단위 연령대 */ 
                               , CASE WHEN T3.SEX_FG_CD IN ('01','02') THEN T3.SEX_FG_CD
                                      ELSE '99' /* 남자:01, 여자:02, 예외:99 */
                                 END AS SEX                                                                                                             /* 성별코드 */
                               -- T1 : 상품추천 INPUT테이블
                                FROM (select distinct intg_cust_no  
                                      FROM LMDAPT3_DB.F_ANL_PROD_RECOM_APPLY
                                      WHERE anl_dt  = '{target_date}') AS T1   
                               -- T3 : 고객기본속성 테이블 
                                LEFT OUTER JOIN lmdapt3_db.D_CUST_BASE_PROP AS T3      /* D_고객기본속성 */
                                  ON T1.INTG_CUST_NO = T3.CUST_NO
                                 ) as A
                          -- B : 연령코드마스터 
                            left outer join lmdapt3_db.d_age_grp_5 B on A.AGE_GRP = B.age_grp_5_cd  
                          ;
                          ''')

# 소분류 시즌상품 마스터 로드

In [62]:
season_prod_df = rs_connect.query_request(f''' 
                                        WITH TA AS (
                                            SELECT T1.STDRD_DATE
                                                 , T4.L2_NM
                                                 , T2.L3_CD
                                                 , T3.L3_NM
                                                 , T1.STR_CD
                                                 , SUM(T1.FINAL_QTY) AS SUM_QTY
                                            FROM LMDAPT2_DB.SA_STR_PROD_STOC_INFO T1
                                            INNER JOIN LMDAPT3_DB.D_PROD_MST T2 ON T1.PROD_CD = T2.PROD_CD
                                            INNER JOIN LMDAPT3_DB.D_PROD_L3 T3 ON T2.L3_CD = T3.L3_CD
                                            INNER JOIN LMDAPT3_DB.D_PROD_L2 T4 ON T2.L2_CD = T4.L2_CD
                                            WHERE 1=1 
                                            AND T1.FINAL_QTY >= 0 
                                            GROUP BY 1,2,3,4,5
                                            ),
                                        TB AS (SELECT STDRD_DATE
                                             , L2_NM
                                             , L3_CD
                                             , L3_NM
                                             , SUM(SUM_QTY) ALL_QTY
                                             , AVG(SUM_QTY) AVG_QTY
                                             FROM TA
                                             GROUP BY 1,2,3,4
                                        ),    
                                        TC AS (SELECT TA.STDRD_DATE
                                             , TA.L2_NM
                                             , TA.L3_CD
                                             , TA.L3_NM
                                             , TB.ALL_QTY
                                             , TB.AVG_QTY
                                              , CAST(COUNT(DISTINCT CASE WHEN TA.SUM_QTY > 0 THEN TA.STR_CD ELSE NULL END) AS DECIMAL)  / 120.0 AS STR_RT_0
                                              , CAST(COUNT(DISTINCT CASE WHEN TA.SUM_QTY >= 3 THEN TA.STR_CD ELSE NULL END) AS DECIMAL) / 120.0 AS STR_RT_3
                                              , CAST(COUNT(DISTINCT CASE WHEN TA.SUM_QTY >= 5 THEN TA.STR_CD ELSE NULL END) AS DECIMAL) / 120.0 AS STR_RT_5
                                              , CAST(COUNT(DISTINCT CASE WHEN TA.SUM_QTY >= 10 THEN TA.STR_CD ELSE NULL END) AS DECIMAL)/ 120.0 AS STR_RT_10          
                                              , CAST(COUNT(DISTINCT CASE WHEN TA.SUM_QTY >= TB.AVG_QTY THEN TA.STR_CD ELSE NULL END) AS DECIMAL) / 120.0 STR_RT_AVG
                                         FROM TA
                                         INNER JOIN TB 
                                         ON TA.L3_NM = TB.L3_NM
                                        GROUP BY 1,2,3,4,5,6),
                                        TD AS (SELECT *,SYSDATE AS LOAD_DT
                                               FROM LMDAPWK_DB.PD_PROD_SEASON_MST T1
                                        ) 
                                        SELECT TD.L3_CD,TD.SEASON_STR_MONTH,TD.SEASON_END_MONTH,TD.SEASOM_MONTH_CNT,TC.STR_RT_3 
                                        FROM TD 
                                        LEFT JOIN TC ON TC.L3_CD = TD.L3_CD 
                                        ;

''')

In [63]:
def get_index(x):
    end_date = (datetime.datetime.strptime(x['SEASON_START_YM'] , '%Y%m') + relativedelta(months=x['SEASOM_MONTH_CNT_LAST'])).strftime('%Y%m')
    return end_date

season_prod_df['SEASOM_MONTH_CNT_LAST'] = season_prod_df['SEASOM_MONTH_CNT']-1
season_prod_df['SEASON_START_YM']  = target_date[0:4]+season_prod_df['SEASON_STR_MONTH']
season_prod_df['SEASON_END_YM'] = season_prod_df.apply(get_index,axis=1)

In [64]:
season_prod_df

L3_CD SEASON_STR_MONTH SEASON_END_MONTH  SEASOM_MONTH_CNT  STR_RT_3  \
0   001001002               05               08                 4    0.6083   
1   001001013               09               01                 5    0.9583   
2   001001006               06               08                 3    0.0083   
3   001001022               10               02                 5    0.9583   
4   001001012               12               03                 4    0.5583   
5   001001017               04               06                 3    0.0000   
6   001001031               08               08                 1    0.0000   
7   001001001               05               09                 5    0.8666   
8   001001008               08               09                 2    0.0000   
9   001001025               08               10                 3    0.0083   
10  001001027               10               12                 3    0.4416   
11  001001004               06               08                 3    0.0083   
12  001001026               06               07                 2    0.0000   
13  001001015               11               04                 6    0.9333   
14  001001029               06               06                 1    0.0000   
15  001001016               05               07                 3    0.0083   
16  001001003               03               07                 5    0.0083   
17  001001007               08               10                 3    0.9583   
18  001001024               05               06                 2       NaN   

    SEASOM_MONTH_CNT_LAST SEASON_START_YM SEASON_END_YM  
0                       3          202205        202208  
1                       4          202209        202301  
2                       2          202206        202208  
3                       4          202210        202302  
4                       3          202212        202303  
5                       2          202204        202206  
6                       0          202208        202208  
7                       4          202205        202209  
8                       1          202208        202209  
9                       2          202208        202210  
10                      2          202210        202212  
11                      2          202206        202208  
12                      1          202206        202207  
13                      5          202211        202304  
14                      0          202206        202206  
15                      2          202205        202207  
16                      4          202203        202207  
17                      2          202208        202210  
18                      1          202205        202206

## Step9) 최종 소분류 추천결과 생성 

In [ ]:
def Last_df_filtering(last_df,cut_off):
    last_df.columns = [i.upper() for i in last_df.columns]
    last_cols = ['ANL_DT','INTG_CUST_NO','SELL_TYPE','L3_CD','L3_NM','MRR','L3_RANK','HYPER_TEAM_CD','HYPER_TEAM_NM','L1_RANK']
    last_df = pd.merge(last_df,cust_demo_df[['INTG_CUST_NO','AGE_GRP','SEX','AGE_GRP_5_NM']],how='left',on=['INTG_CUST_NO'])
    # None AGE GRP & 20세미만 청소년 주류팀 제거
    last_df = last_df.drop(last_df[(last_df.AGE_GRP.isin(['01','99'])) &(last_df['HYPER_TEAM_CD'].isin([EXCEP_ALCOHOL_TEAM_CD])) ].index)    
    re_df = last_df[(last_df['SELL_TYPE']=='RE-SELL')]
    cross_df = last_df[(last_df['SELL_TYPE']=='CROSS-SELL')]
    cross_df = cross_df[cross_df['L1_RANK'] <=cut_off]
    last_df2 = pd.concat([re_df,cross_df])
    last_df2.loc[:,'L3_RANK'] = last_df2.groupby(['INTG_CUST_NO','SELL_TYPE'])['MRR'].rank(method='dense',ascending=False).astype(int)
    last_df2 = last_df2.loc[(last_df2['L3_RANK'] <= 60),last_cols]
    last_df2 = last_df2[last_cols].sort_values(by=['INTG_CUST_NO','SELL_TYPE','L3_RANK'],ascending=[True,True,True])
    last_df2 = last_df2[~last_df2['INTG_CUST_NO'].isin(['99999999999999'])]
    return last_df2

cs_rm_df    = rs_connect.query_request('''SELECT * FROM lmdapt2_db.pd_cross_sell_excpt_prod_list''')
cs_rm_list  = cs_rm_df.L3_CD.unique()
# logger_success.error(f'''*** Original Last_df shape : {last_df.shape}''')
last_df = last_df.drop(last_df[(last_df['SELL_TYPE']=='CROSS-SELL')&(last_df['L3_CD'].isin(cs_rm_list))].index)
#     print(f'filtering cross-sell list Last_df shape : {last_df.shape}')
# logger_success.error(f'''*** filtering cross-sell list Last_df shape : {last_df.shape}''')

last_df2 = Last_df_filtering(last_df,4)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step8) Make last result table & filter cust complete.. ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")

# Step10) 소분류 추천결과 Output폴더에 .chk파일과 적재

In [ ]:
new_cust_list = tuple(last_df2.INTG_CUST_NO.unique().tolist())
tbl_name = 'F_ANL_PROD_RECOM_L3_RESULT'

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step9)  F_ANL_PROD_RECOM_L3_RESULT  ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
#     rs_connect.replace_table(last_df2,'lmdapwk_anl_db.F_ANL_PROD_RECOM_L3_RESULT',True)
log_time = np.round((time.time() - start_time) / 60, 2)

path = '/analytics/mdap/src/data/output/'
output_file = f'F_ANL_PROD_RECOM_L3_RESULT_{target_date_6digits}'

# if os.path.isfile(path+output_file+'.csv'):
#     os.remove(path+output_file+'.csv')
# if os.path.isfile(path+output_file+'.chk'):
#     os.remove(path+output_file+'.chk')

last_df2[['ANL_DT','INTG_CUST_NO','SELL_TYPE','L3_CD','L3_NM','L3_RANK']].to_csv(f'{path+output_file}.csv',index=0,encoding='utf-8')
chk_file = f'{path+output_file}.chk'
open(chk_file, mode='w').close()

# cl.move_data()
# cl.remove_data()
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    " || Step10) Prod recom L3 Result File Upload complete..!! ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")
log_time = np.round((time.time() - start_time) / 60, 2)

# Step11) S3 버킷 비우기

In [10]:
# rm_all_bucket.delete_all_object_in_bucket(BUCKET_NAME)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    f" || Step11) Delete all in '{BUCKET_NAME}' S3 bucket complete..!! ",
    "(elapsed time :", np.round((time.time() - start_time) / 60, 2), "mins)")

Emptied bucket : lottemart-prd-adm-s3
2022-09-21 15:29:44  || Step11) Delete all in 'lottemart-prd-adm-s3' S3 bucket complete..!!  (elapsed time : 47.54 mins)
